In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
%matplotlib inline

In [3]:
df_twitter = pd.read_csv('data/twcs.csv')
print('Twitter Data: ' + str(df_twitter.shape))

Twitter Data: (2811774, 7)


In [4]:
# Twitter Dataset Sort by Date
# df_twitter.columns
# df_twitter.dtypes
# df_twitter["created_at"] = pd.to_datetime(df_twitter["created_at"], utc=True)
# df_twitter.dtypes
#df_twitter = df_twitter.sort_values(by="created_at")

In [5]:
unique_comp = pd.DataFrame(df_twitter['author_id'].unique())

In [6]:
# Get earliest date
df_twitter.head(n=5)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [7]:
# Get latest date
df_twitter.tail(n=1)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
2811773,2987950,823870,True,Tue Nov 21 22:01:04 +0000 2017,@AldiUK warm sloe gin mince pies with ice cre...,"2987951,2987949",NaN


In [8]:
# sns.countplot(x='author_id', data=df_twitter)

### Cleaning the Data

In [25]:
# Making all tweets lowercase
df_twitter['text'] = df_twitter["text"].str.lower()

# initializing stop words
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))
nltk.download('punkt')

# initializing punctuations string
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~‘'''


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\carol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
# Creating function to create dataframe for each tweet, eliminating stop words and punctuation
# Please note the column name is the tweet_id name

df_tweets = pd.DataFrame()

def twitter_tweets(number_of_rows):
    for x in range(number_of_rows):
        tweet_filtered = []
        words_in_tweet = word_tokenize(df_twitter['text'][x])
        for w in words_in_tweet:
            if w not in stop_words:
                if w not in punc:
                    tweet_filtered.append(w)
        df_tweets[df_twitter['tweet_id'][x]] = pd.Series(tweet_filtered)

twitter_tweets(2811774)
df_tweets.to_csv('tweets.csv')


C:\Users\carol\AppData\Local\Temp/ipykernel_13092/3344256597.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tweets[df_twitter['tweet_id'][x]] = pd.Series(tweet_filtered)


In [11]:
# Transpose the Twitter Dataframe for analysis

df_tweets_transpose = df_tweets.T

# See the first ten rows of the transpose
df_tweets_transpose.head(n=10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
1,115712,I,understand,I,would,like,assist,We,would,need,get,private,secured,link,assist
2,115712,I,understand,I,would,like,assist,We,would,need,get,private,secured,link,assist
3,115712,I,understand,I,would,like,assist,We,would,need,get,private,secured,link,assist
4,115712,I,understand,I,would,like,assist,We,would,need,get,private,secured,link,assist
5,115712,I,understand,I,would,like,assist,We,would,need,get,private,secured,link,assist
6,115712,I,understand,I,would,like,assist,We,would,need,get,private,secured,link,assist
8,115712,I,understand,I,would,like,assist,We,would,need,get,private,secured,link,assist
11,115712,I,understand,I,would,like,assist,We,would,need,get,private,secured,link,assist
12,115712,I,understand,I,would,like,assist,We,would,need,get,private,secured,link,assist
15,115712,I,understand,I,would,like,assist,We,would,need,get,private,secured,link,assist
